In [1]:
#importing the tools
import numpy as np
import pandas as pd
import hyperopt
from catboost import Pool, CatBoostClassifier, cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
#Reading the Dataset
train = pd.read_csv("DSN_train.csv")
test = pd.read_csv("DSN_test.csv")

In [4]:
train.isnull().sum()

EmployeeNo                                0
Division                                  0
Qualification                          1679
Gender                                    0
Channel_of_Recruitment                    0
Trainings_Attended                        0
Year_of_birth                             0
Last_performance_score                    0
Year_of_recruitment                       0
Targets_met                               0
Previous_Award                            0
Training_score_average                    0
State_Of_Origin                           0
Foreign_schooled                          0
Marital_Status                            0
Past_Disciplinary_Action                  0
Previous_IntraDepartmental_Movement       0
No_of_previous_employers                  0
Promoted_or_Not                           0
dtype: int64

In [5]:
#Data Cleaning by replacing missing values
train.fillna(-999,inplace=True)
test.fillna(-999,inplace=True)

In [6]:
train.isnull().sum()

EmployeeNo                             0
Division                               0
Qualification                          0
Gender                                 0
Channel_of_Recruitment                 0
Trainings_Attended                     0
Year_of_birth                          0
Last_performance_score                 0
Year_of_recruitment                    0
Targets_met                            0
Previous_Award                         0
Training_score_average                 0
State_Of_Origin                        0
Foreign_schooled                       0
Marital_Status                         0
Past_Disciplinary_Action               0
Previous_IntraDepartmental_Movement    0
No_of_previous_employers               0
Promoted_or_Not                        0
dtype: int64

In [7]:
x = train.drop('Promoted_or_Not',axis=1)
y = train.Promoted_or_Not

In [8]:
x.dtypes

EmployeeNo                              object
Division                                object
Qualification                           object
Gender                                  object
Channel_of_Recruitment                  object
Trainings_Attended                       int64
Year_of_birth                            int64
Last_performance_score                 float64
Year_of_recruitment                      int64
Targets_met                              int64
Previous_Award                           int64
Training_score_average                   int64
State_Of_Origin                         object
Foreign_schooled                        object
Marital_Status                          object
Past_Disciplinary_Action                object
Previous_IntraDepartmental_Movement     object
No_of_previous_employers                object
dtype: object

In [9]:
print(x.shape)
print(y.shape)

(38312, 18)
(38312,)


In [10]:
#declaration of categorical features
cate_features_index = np.where(x.dtypes != float)[0];cate_features_index

array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
      dtype=int64)

In [20]:
#Applying train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.30, random_state=42)

In [30]:
#importing Machine Learning Algorithms
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
import sklearn.metrics as sklm
import xgboost as xgb
from catboost import Pool, CatBoostClassifier, cv
from sklearn.metrics import accuracy_score

In [32]:
#Training the model
model = CatBoostClassifier(iterations=3000, depth=8, learning_rate=0.04,eval_metric='Accuracy',use_best_model=True, random_seed=42)
model.fit(xtrain,ytrain,cat_features=cate_features_index,eval_set=(xtest,ytest))

0:	learn: 0.9155045	test: 0.9151731	best: 0.9151731 (0)	total: 139ms	remaining: 6m 55s
1:	learn: 0.9379521	test: 0.9399687	best: 0.9399687 (1)	total: 1.3s	remaining: 32m 36s
2:	learn: 0.9390335	test: 0.9401427	best: 0.9401427 (2)	total: 2.12s	remaining: 35m 13s
3:	learn: 0.9392199	test: 0.9401427	best: 0.9401427 (2)	total: 2.52s	remaining: 31m 30s
4:	learn: 0.9394809	test: 0.9399687	best: 0.9401427 (2)	total: 3.04s	remaining: 30m 22s
5:	learn: 0.9394437	test: 0.9397947	best: 0.9401427 (2)	total: 3.59s	remaining: 29m 50s
6:	learn: 0.9398538	test: 0.9397947	best: 0.9401427 (2)	total: 4.49s	remaining: 32m
7:	learn: 0.9395182	test: 0.9399687	best: 0.9401427 (2)	total: 4.74s	remaining: 29m 32s
8:	learn: 0.9394437	test: 0.9399687	best: 0.9401427 (2)	total: 5.63s	remaining: 31m 11s
9:	learn: 0.9395182	test: 0.9399687	best: 0.9401427 (2)	total: 6.04s	remaining: 30m 5s
10:	learn: 0.9394809	test: 0.9399687	best: 0.9401427 (2)	total: 7.2s	remaining: 32m 37s
11:	learn: 0.9395555	test: 0.9399687	be

In [23]:
#Using Catboost Algorithm
import sklearn.metrics as sklm

In [24]:
#declarationtion of test parameters
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])

def print_metrics(labels, probs, threshold):
    scores = score_model(probs, threshold)
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    print('AUC             %0.2f' % sklm.roc_auc_score(labels, probs[:,1]))
    print('Macro precision %0.2f' % float((float(metrics[0][0]) + float(metrics[0][1]))/2.0))
    print('Macro recall    %0.2f' % float((float(metrics[1][0]) + float(metrics[1][1]))/2.0))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])
    
probabilities = model.predict_proba(xtest)
print_metrics(ytest, probabilities, 0.5)   

                 Confusion matrix
                 Score positive    Score negative
Actual positive     10501                18
Actual negative       649               326

Accuracy        0.94
AUC             0.90
Macro precision 0.94
Macro recall    0.67
 
           Positive      Negative
Num case    10519           975
Precision    0.94          0.95
Recall       1.00          0.33
F1           0.97          0.49


In [25]:
#Predicting the outcome of the loan application
solution=model.predict((test))
np.savetxt('DSN_Final.csv', solution, delimiter=',')
my_submission=pd.DataFrame({'EmployeeNo':test.EmployeeNo,'Promoted_or_Not': solution})
my_submission.to_csv('Proj.csv', index=False)

In [26]:
#Printing the accuracy
print('the test accuracy is :{:.6f}'.format(accuracy_score(ytest, model.predict(xtest))))

the test accuracy is :0.941970


In [27]:
#predicting the test variable
pred = model.predict(test)
pred = pred.astype(np.int)
submission = pd.DataFrame({'EmployeeNo':test['EmployeeNo'],'Promoted_or_Not':pred})

In [29]:
#Saving the prediction outcome as scv file
submission.to_csv('DSN_final_13.csv',index=False)